# Descripción de la Actividad Evaluativa
Vamos a crear un sistema de facturación y bodega simplificado, el cual tendrá diferentes opciones:

a) Ingresar producto: debe ingresarse el nombre y el precio del producto. Tendrá un sub menú: nuevo producto, guardar archivo o volver.

b) Ingresar factura: debe ingresarse el nombre del cliente, luego deberá generar un submenú: ingresar producto, guardar y salir.

Los productos se ingresan con nombre y precio. La factura además debe guardar la fecha y hora actual de la venta.

c) Ver resumen: en esta opción se debe desplegar:
Cantidad de ventas y monto total vendido. Opción de guardar resumen (ingresando nombre de archivo).

d) Ver detalle: se debe tener las opciones desplegar: ventas por usuario, ventas por producto, ventas diarias, guardar vista (ingresando el nombre de archivo).

# Importacion de Librerias Panda

In [ ]:
import pandas as pd
from datetime import datetime #importacion de modulo datetime

archivo_productos="productos.csv" #se crean los nombres de los archivos a desplegar
archivo_facturas="facturas.csv"
archivo_detalle="detalle.csv"


Carga de archivos

Execpciones de errores

In [ ]:
try:
    df_productos= pd.read_csv(archivo_productos) #libreria lee el archivo
except FileNotFoundError: #exepcion de error
    df_productos=pd.DataFrame(columns=['Nombre','Precio'])#almacena datos en las columnas nombre y precio

try:
    df_facturas=pd.read_csv(archivo_facturas) #libreria lee el archivo
except FileNotFoundError: #exepcion de error
    df_facturas=pd.DataFrame(columns=['Cliente', 'Fecha', 'Productos']) #almacena datos en las columnas cliente, fecha y productos


a) Ingresar Producto

In [ ]:
def ingresar_producto(): #metodo ingresar producto
    global df_productos #variable global
    nombre = input("Ingrese el nombre del producto:") #se almacena el nombre del producto en la variable nombre

    while True: #bucle de error
        precio_input=input("ingrese el precio del producto") # se almacena el dato indicado por el usuario en la varible
    try:
        precio = float(precio_input) #se cambia la variable a tipo flotante
         # break
    except ValueError:
        print("Error: Ingrese un valor numerico valido") #se despliega mensaje de error

    nuevo_producto = pd.DataFrame({'Nombre':[nombre],'Precio': [precio]}) #funcion almacena datos en columnas nombre y precio
    df_productos=pd.concat([df_productos,nuevo_producto],ignore_index=True) #funcion que une las dataframe

    print("producto ingresado correctamente") #mensaje desplegado cuando se ingresa un producto


b) Ingresar Factura

In [ ]:
def ingresar_factura():#metodo ingresar factura
    global df_facturas #varible global
    cliente=input("Ingrese el nombre del cliente: ")#se almacena el nombre del cliente en la variable
    fecha = datetime.now().strftime('%y-%m-%d %H:%M:%S') #se registra la fecha actual, con la funcion .now en el formato aa/mm/dd h:m:s

    productos_factura = [] #se crea lista vacia
    while True: #bucle
        nombre_producto = input("Ingrese el nombre del producto(o'q' para salir):") #se almacena el nombre del producto en la variable. o puede salir del precionando q
        if nombre_producto.lower()=='q': #si el usuario ingreso Q, la cambia a minuscula con la funcion .lower
           break
        productos_factura.append(nombre_producto) #la funsion append agrega el nombre ingresado por el usuario, a la ultima posicion de la cola

    nueva_factura = pd.DataFrame({'Cliente':[cliente], 'Fecha':[fecha], 'Productos': [productos_factura]})#se definen los dataframe de la factura
    df_facturas = pd.concat([df_facturas, nueva_factura],ignore_index=True) #funcio que une las dataframe
    print("Factura ingresada correctamente")#mensaje desplegado al almacenar los datos ingresados por el usuario


c) Ver Resumen

In [ ]:
def ver_resumen():#metodo
    global df_facturas, df_productos #variable global
    cantidad_ventas = len(df_facturas)#la variable sera el numero de valores de la lista de facturas almacenadas
    monto_total_vendido = df_facturas.apply(lambda row: sum(df_productos[df_productos['Nombre'].isin(row['Productos'])]['Precio']),axis=1).sum()#calculo para determinar el monto total de facturas, usando la funcion lambda

    print(f"\nResumen:") #Titulo desplegado
    print(f"cantidad de ventas: {cantidad_ventas}") #despliega la cantidad de ventas
    print(f"monto total vendido: {monto_total_vendido}") #despliega el monto total vendido

    opcion_guardar = input("¿Desea guardar el resumen? (S/N): ") #pregunta al usuario si desea guardar el resumen
    if opcion_guardar.lower() =='s':#cambia la S a minuscula
       archivo_guardar = input("ingrese el nombre del arcivo para guardar el resumen: ") # si el usuario desea guardar el archivo, se pide ingresa el nombre del archivo
       df_resumen = pd.DataFrame({'cantidad_ventas': [cantidad_ventas], 'Monto_Total_Vendido': [monto_total_vendido]})# las dataframes se almacenan en la lista
       df_resumen.to_csv(archivo_guardar + '.csv', index=False)#la lista con las dataframes se convierte en archivo formato .csv
       print(f"resumen guardado en {archivo_guardar}.csv") #mensaje de confirmacion que fue guardado el archivo


d) Ver detalle

In [ ]:
def ver_detalle(): #metodo ver detalle
    global df_facturas #variable global
    print("\nDetalle: ")#Titulo del menu
    print("1. ventas por usuario") #lista de opciones para elegir
    print("2. Ventas por producto")
    print("3. ventas diarias")
    print("4. volver")

    opcion_detalle = input("seleccione una opcion: ") #ingreso de opcion

    if opcion_detalle =='1': #opcion ventas por usuario
       ventas_usuario = df_facturas.groupby('Cliente').size().reset_index(name='cantidad_ventas')#despliega la cantidad de ventas realizadas por un usuario
      #la funcion groupby divide la dataframe para rescatar la informacion del cliente y de la cantidad de ventas realizas
       print(ventas_usuario) #despliega el listado de cantidad de ventas por usuario
    elif opcion_detalle=='2':#opcion ventas por producto
         ventas_producto=df_facturas.explode('Productos')#la funcion explode divide del dataframe a productos
         ventas_producto=ventas_producto.groupby('Productos').size().reset_index(name='cantidad_ventas') #
         #la funcion groupby divide la dataframe para rescatar la informacion de productos y de la cantidad de ventas realizas
         print(ventas_producto) # despliega el listado de ventas por producto
    elif opcion_detalle == '3':#opcion 3 ventas diarias
         ventas_diarias =df_facturas.groupby(df_facturas['Fecha'].str.split(' ').str[0]).size().reset_index(name='cantidad-ventas')
         #la funcion groupby divide la dataframe para rescatar la informacion de la cantidad de ventas realizas en la fecha
         print(ventas_diarias) #despliega listado de ventas con fecha
    elif opcion_detalle =='4':#opcion volver
          return

    opcion_guardar =input("¿Desea guardar la vista? (s\n):") #Mensaje para usuario si desea guardar la informacion desplegada
    if opcion_guardar.lower()== 's':#cambia de S a minuscula
       archivo_guardar =input("Ingrese el nombre del archivo para guardar la vista: ") #solicita el nombre del archivo para guardar
       if opcion_detalle == '1':
          ventas_usuario.to_csv(archivo_guardar + '.csv', index=False) #la lista con las dataframes se convierte en archivo formato .csv
       elif opcion_detalle=='2':
            ventas_producto.to_csv(archivo_guardar + 'csv', index=False)#la lista con las dataframes se convierte en archivo formato .csv
       print(f"vista guardada en {archivo_guardar}.csv") #mensaje de confirmacion que se guardo el archivo


e) Borrar datos

In [ ]:
def borrar_datos(): #metodo para borrar datos
    global df_productos,df_facturas #variables globales
    print("\nBorrar datos: ") #Titulo
    print("1. Borrar productos")#opciones
    print("2. Borrar facturas")
    print("3. Borrar todos los datos")
    print("4. volver")

    opcion_borrar = input("Seleccione una opcion: ") #mensaje solicita ingresar una opcion, la cual se almacena en la varible

    if opcion_borrar =='1': #opcion borrar
       df_productos = pd.DataFrame(columns=['Nombre', 'Precio']) #se modifican las columnas del dataframe
       df_productos.to_csv(archivo_productos, index=False) # se crea un archivo .csv con el dataframe
       print("Datos de productos borrados.") #mensaje de confirmacion de borrado de datos
    elif opcion_borrar =='2':
        df_facturas=pd.DataFrame(columnas=['cliente','fecha','productos'])#se modifican las columnas del dataframe
        df_facturas.to_csv(archivo_facturas, index=False) # se crea un archivo .csv con el dataframe
        print("Datos de facturas borrados.")#mensaje de confirmacion de borrado de datos
    elif opcion_borrar =='4':#opcion volver
        return

f) Salir

In [ ]:
def main(): #ejecucion del algoritmo
    global df_productos, df_facturas #variables globales
    while True: #bucle condicional
         print("\nSistema de facturacion y bodega")#titulo del menu
         print("\nMenu principal:")#Menu
         print("\na. Ingresar Producto\nb. Ingresar Factura\nc. Ver Resumen\nd. Ver Detalle\ne. Borrar Datos\nf. Salir") #mensaje de opciones
         opcion =input("\nSeleccione una opcion: ").lower() #se almacena la opcion ingresada y si es mayuscula la cambia a minuscula
         if opcion=='a':
             ingresar_producto() #llama al metodo ingresar producto
         elif opcion =='b':
             ingresar_factura()#llama al metodo ingresar factura
         elif opcion=='c':
             ver_resumen()#llama al metodo ver resumen
         elif opcion =='d':
             ver_detalle()#llama al metodo ver detalle
         elif opcion =='e':
             borrar_datos()#llama al metodo borrar datos
         elif opcion=='f':
              print("saliendo") #mensaje confirma salida de algoritmo
              break
         else:
              print("Opcion no valida. Intentelo de nuevo.")#mensaje que despliega el ingreso de una opcion no valida

if __name__=="__main__":
    main()